<p align="left">
  <img src="./logo_UTN.svg" />
</p>


# **Teoría de Circuitos II - R4001 - 2023**
## Trabajo Semanal 4
### Autor: Bruno Glecer

# Consigna

La tarea semanal consiste en diseñar un filtro pasa-banda con las siguientes caracteristicas

- Frecuencia de corte inferior $f_{ci} = 1600\mathrm{kHz}$
- Frecuencia de corte superior $f_{cs} = 2500\mathrm{kHz}$
- Ripple maximo en la banda de paso: $3\mathrm{dB}$
- Maxima planicidad en la banda de paso
- Ganancia maxima en la banda de paso: $10\mathrm{dB}$
- Atenuacion minima de $\alpha_{min} = 20\mathrm{dB}$ a las frecuencias $1250\mathrm{kHz}$ y $3200\mathrm{kHz}$
Se piden las siguientes consignas:

Se pide:

1. Obtener la funcion de transferencia normalizada del filtro
2. Graficar el diagrama de polos y ceros
3. Graficar la transferencia (modulo y fase) del filtro pedido
4. Sintetizar el filtro utilizando estructuras Ackeberg-Mossberg
5. Simular el filtro obtenido, verificando las especificaciones de diseño.



# 1. Transferencia

Comenzamos normalizando los parametros del filtro usando la frecuencia central $\omega_0 = \sqrt{\omega_{ci} \omega_{cs}}$ como norma para diseñar el pasabajos prototipo y luego aplicar la transformacion en frecuencia utilizando el nucleo $p = K(s) = Q\dfrac{s^2 + 1}{s}$

In [20]:
import numpy as np

#Uso los nombres de f_1 y f_2 para las frecuencias de paso y f_s1 y f_s2 para las frecuencias de corte

f_p1 = 1600*10**3
f_p2 = 2500*10**3

f_s1 = 1250*10**3
f_s2 = 3200*10**3

alpha_max = 3
alpha_min = 20

#Caluclos de los parametros del filtro

f_0 = np.sqrt(f_p1*f_p2)
bw = f_p2 - f_p1
q_filtro = f_0/bw

#Paso a frecuencia angular y normalizo

w_norma = f_0 * 2*np.pi

w_0  = 1

w_p1 = f_p1 * 2*np.pi / w_norma
w_p2 = f_p2 * 2*np.pi / w_norma

w_s1 = f_s1 * 2*np.pi / w_norma
w_s2 = f_s2 * 2*np.pi / w_norma

print(f"Frecuencias de paso: ({w_p1} a  {w_p2})")
print(f"Frecuencias de corte: ({w_s1}  a  {w_s2})")


Frecuencias de paso: (0.7999999999999999 a  1.2500000000000002)
Frecuencias de corte: (0.6250000000000001  a  1.5999999999999999)


Para obtener los parametros del filtro prototipo que debemos diseñar, tenemos que elegir los valores de frecuencia tal que imponga las restricciones mas fuertes en el caso de que la plantilla no sea simetrica.

In [5]:
#Transformo los parametros del filtro pasabanda al filtro prototipos usando el nucleo de transformacion:

def nucleo_wi(w):
    return q_filtro*((w**2 - 1)/w) #El nucleo es diferente al definido porque esta formula es para transformar el valor absoluto de frecuencia imaginaria, no compleja.

w_p1_lp = nucleo_wi(w_p1)
w_p2_lp = nucleo_wi(w_p2)

w_s1_lp = nucleo_wi(w_s1)
w_s2_lp = nucleo_wi(w_s2)


print(f"Opciones de frecuencia de paso prototipo: ({w_p1_lp} ,  {w_p2_lp})")
print(f"Opciones de frecuencia de corte prototipo: ({w_s1_lp}  a  {w_s2_lp})")



Opciones de frecuencia de paso prototipo: (-1.0000000000000004 ,  1.0000000000000007)
Opciones de frecuencia de corte prototipo: (-2.166666666666666  a  2.1666666666666665)


Seguimos eligiendo los valores de frecuencias para el pasa bajos prototipo de forma que ponga la mayor exigencia posible

In [7]:
w_p_lp = min(abs(w_p1_lp), abs(w_p2_lp))
w_s_lp = min(abs(w_s1_lp), abs(w_s2_lp))

print(f"Frecuencia de paso prototipo: {w_p_lp}")
print(f"Frecuencia de corte prototipo: {w_s_lp}")


Frecuencia de paso prototipo: 1.0000000000000004
Frecuencia de corte prototipo: 2.166666666666666


Como es de esperar, la frecuencia de paso resulta igual a 1 debido a la normaizacion inicial. El filtro prototipo se debe diseñar para que el ripple maximo en la banda de paso sea de $\alpha_{max} = 3\mathrm{dB}$ y una atenuacion en la frecuencia de corte minima de $\alpha_{min} = 20dB$. Pasamos entonces a calcular los parametros del filtro prototipo



In [12]:
#Calculo de epsilon

epsilon = np.sqrt(10**(alpha_max/10) - 1)

#Calculo de orden

n = 1

while (10*np.log10(1 + epsilon**2 * w_s_lp ** (2*n))) < alpha_min:
    n += 1
    
alpha_ws = 10*np.log10(1 + epsilon**2 * w_s_lp ** (2*n))

    
print(f"epsilon = {epsilon}    n = {n}   alpha_ws = {alpha_ws}")

epsilon = 0.9976283451109834    n = 3   alpha_ws = 20.168877004136377


El filtro al tener una atenuacion maxima en la banda de paso de 3dB, corresponde a un filtro Butterworth ($\varepsilon=1$), entonces no es necesario desnormalizar con la frecuencia de Butterworth.

Continuamos calculando los polos del filtro utilizando la propiedad de separacion angular constante entre los polos de un filtro de maxima planicidad.

$T_{LP}(p) = \dfrac{1}{\left(p + 1\right) \left(p^2 + 2 cos \left(\dfrac{\pi}{3}\right)p + 1 \right)} = \dfrac{1}{p^3 + 2p^2 + 2p + 1}$

Remplazando $p$ por el nucleo de transformacion, podemos encontrar la transferencia del filtro pasabanda



In [59]:
import scipy.signal
from IPython.display import display
from pytc2.sistemas_lineales import analyze_sys, pretty_print_bicuad_omegayq,tfcascade,pretty_print_lti

t = signal.butter(3, [w_p1, w_p2], btype='bandpass', analog=True)

tlp = signal.butter(3, 1, btype='lowpass', analog=True)
t2 = signal.lp2bp(tlp[0], tlp[1], 1, 0.45)

pretty_print_lti(t[0], t[1])
pretty_print_lti(t2[0], t2[1])
print(t2)
print( [w_p1, w_p2])

<IPython.core.display.Math object>

<IPython.core.display.Math object>

(array([0.091125, 0.      , 0.      , 0.      ]), array([1.      , 0.9     , 3.405   , 1.891125, 3.405   , 0.9     ,
       1.      ]))
[0.7999999999999999, 1.2500000000000002]


In [51]:
from sympy.abc import p, s, T
from IPython.display import display
import sympy

def round_expr(expr, num_digits):
    return expr.xreplace({n : round(n, num_digits) for n in expr.atoms(sympy.Number)})

transferencia_lp = sympy.Eq(T, sympy.Poly([1], p)/sympy.Poly([1, 2, 2, 1], p))
relacion_p_s = sympy.Eq(p, q_filtro * (s**2 + 1)/s)

transferencia = sympy.solve([transferencia_lp, relacion_p_s], [T, p], dict=True)[0][T]

#numerador = sympy.fraction(transferencia, s)[0]
#denominador = sympy.fraction(transferencia, s)[1]

#denominador_coef_principal = denominador.coeff(s,6)

#numerador /= denominador_coef_principal
#denominador /= denominador_coef_principal

#transferencia = numerador/denominador

display(round_expr(transferencia, 5))

729.0*s**3/((20.0*s**2 + 9.0*s + 20.0)*(400.0*s**4 + 180.0*s**3 + 881.0*s**2 + 180.0*s + 400.0))